# 实验 5.1：标签翻转攻击

## 实验目标
- 理解什么是**标签翻转攻击**，以及它为什么属于数据投毒
- 实现一个简单的标签翻转投毒函数
- 观察不同投毒比例对模型准确率的影响
- 从防守视角思考：为什么这种攻击容易实现、也相对容易被发现

## 📚 前置知识
- 了解机器学习分类任务的基本概念
- 了解神经网络训练的基本流程
- 基础 Python 和 NumPy 操作

## 🖥️ 实验环境
- 平台：腾讯 Cloud Studio
- GPU：NVIDIA Tesla T4（16GB）
- 模型：自定义 PyTorch 二分类网络

## 📝 填空说明
本实验共 **5 个填空**，难度：⭐⭐☆☆☆

⚠️ **安全提醒**：本实验仅用于教育目的，帮助理解数据投毒风险，请勿将技术用于恶意用途。

## 📖 概念回顾

### 什么是标签翻转攻击？

标签翻转攻击是**最简单的数据投毒方式**：攻击者故意给部分训练数据标注错误的标签。

```
正常数据:  图片[猫] → 标签: 猫 ✓
投毒数据:  图片[猫] → 标签: 狗 ✗（标签被翻转）
```

### 攻击原理

| 阶段 | 正常流程 | 攻击流程 |
|------|---------|----------|
| 数据收集 | 收集正确标注的数据 | 混入错误标签的数据 |
| 模型训练 | 学习正确的模式 | 学习到错误的决策边界 |
| 模型预测 | 准确分类 | 容易出错 |

### 为什么有效？

机器学习模型**信任所有训练数据**。即使只有 5-10% 的数据被投毒，模型也会尝试"学习"这些错误，导致决策边界偏移。

## 第一部分：环境准备

In [ ]:
# ====== 环境验证脚本 ======
import sys
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt

# 设置中文显示
plt.rcParams['font.sans-serif'] = ['SimHei', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

# 设置随机种子，确保结果可复现
torch.manual_seed(42)
np.random.seed(42)

print("-" * 50)
print("AI 安全实验环境检查")
print("-" * 50)

# 检查 Python 版本
print(f"\n[1] Python 版本: {sys.version.split()[0]}")

# 检查 PyTorch 和 CUDA
print(f"[2] PyTorch 版本: {torch.__version__}")
print(f"    CUDA 可用: {'✓ 是' if torch.cuda.is_available() else '✗ 否'}")

if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3
    print(f"    GPU 型号: {gpu_name}")
    print(f"    GPU 显存: {gpu_memory:.1f} GB")

print("\n" + "-" * 50)
print("环境准备完成" if torch.cuda.is_available() else "未检测到 GPU（CPU 也可运行，但速度更慢）")
print("-" * 50)

## 第二部分：创建模拟数据集

我们创建一个简单的二维二分类数据集：
- **类别 0**：分布在左下角区域（中心 [-2, -2]）
- **类别 1**：分布在右上角区域（中心 [2, 2]）

这种清晰分离的数据让我们更容易观察投毒的影响。

In [ ]:
def create_dataset(n_samples=1000):
    """
    创建一个简单的二分类数据集
    
    参数:
        n_samples: 样本总数
    
    返回:
        X: 特征数据 (n_samples, 2)
        y: 标签 (n_samples,)
    """
    # 类别0的数据点：中心在(-2, -2)
    X0 = np.random.randn(n_samples // 2, 2) + np.array([-2, -2])
    y0 = np.zeros(n_samples // 2)
    
    # 类别1的数据点：中心在(2, 2)
    X1 = np.random.randn(n_samples // 2, 2) + np.array([2, 2])
    y1 = np.ones(n_samples // 2)
    
    # 合并数据
    X = np.vstack([X0, X1])
    y = np.hstack([y0, y1])
    
    # 打乱顺序
    indices = np.random.permutation(n_samples)
    return X[indices], y[indices]

# 创建训练集和测试集
X_train, y_train = create_dataset(800)
X_test, y_test = create_dataset(200)

print("=" * 50)
print("📊 数据集创建完成")
print("=" * 50)
print(f"  训练集大小: {len(X_train)} 样本")
print(f"  测试集大小: {len(X_test)} 样本")
print(f"  特征维度: {X_train.shape[1]}")
print(f"  类别分布: 类别0 = {int(sum(y_train==0))}, 类别1 = {int(sum(y_train==1))}")

# 可视化原始数据
plt.figure(figsize=(8, 6))
plt.scatter(X_train[y_train==0, 0], X_train[y_train==0, 1], 
            c='blue', label='类别 0', alpha=0.5, s=30)
plt.scatter(X_train[y_train==1, 0], X_train[y_train==1, 1], 
            c='red', label='类别 1', alpha=0.5, s=30)
plt.xlabel('特征 1')
plt.ylabel('特征 2')
plt.title('原始训练数据（干净标签）')
plt.legend()
plt.grid(True, alpha=0.3)
plt.axhline(y=0, color='gray', linestyle='--', alpha=0.3)
plt.axvline(x=0, color='gray', linestyle='--', alpha=0.3)
plt.show()

print("\n✓ 数据可视化完成！")

## 第三部分：实现标签翻转攻击

标签翻转攻击的核心操作：
1. 随机选择一部分样本
2. 将它们的标签翻转（0 → 1 或 1 → 0）

In [ ]:
def poison_labels(y, poison_ratio):
    """
    对标签进行投毒（翻转）
    
    参数:
        y: 原始标签数组
        poison_ratio: 投毒比例（0.0-1.0）
    
    返回:
        y_poisoned: 投毒后的标签数组
        poison_indices: 被投毒样本的索引
    """
    y_poisoned = y.copy()
    n_samples = len(y)
    
    # ========== 填空 1：计算投毒样本数量 ==========
    # 
    # 🎯 任务：根据投毒比例计算需要翻转标签的样本数量
    # 
    # 💡 提示：
    #   - 用总样本数乘以投毒比例
    #   - 使用 int() 转换为整数
    # 
    # 难度：⭐☆☆☆☆
    # 
    # 请将 ___________ 替换为正确的代码
    
    n_poison = ___________  # 期望：如 poison_ratio=0.1, n_samples=800, 则 n_poison=80
    
    # ========== 填空 1 结束 ==========
    
    # 随机选择要翻转的样本索引
    poison_indices = np.random.choice(n_samples, n_poison, replace=False)
    
    # ========== 填空 2：翻转标签 ==========
    # 
    # 🎯 任务：将选中样本的标签翻转（0变1，1变0）
    # 
    # 💡 提示：
    #   - 二分类标签只有 0 和 1
    #   - 用 1 减去原标签即可实现翻转：1-0=1, 1-1=0
    # 
    # 难度：⭐☆☆☆☆
    # 
    # 请将 ___________ 替换为正确的代码
    
    y_poisoned[poison_indices] = ___________  # 期望：标签翻转
    
    # ========== 填空 2 结束 ==========
    
    return y_poisoned, poison_indices

# 测试投毒函数：对10%的数据进行投毒
y_train_poisoned_test, poison_idx_test = poison_labels(y_train, 0.1)

print("=" * 50)
print("🧪 投毒函数测试")
print("=" * 50)
print(f"  总样本数: {len(y_train)}")
print(f"  投毒比例: 10%")
print(f"  投毒样本数: {len(poison_idx_test)}")
print(f"  实际投毒比例: {len(poison_idx_test)/len(y_train)*100:.1f}%")

### ✅ 检查点 1

运行到这里，你应该看到：
- [ ] 投毒样本数为 80（800 × 10%）
- [ ] 实际投毒比例为 10.0%

如果结果不正确，请检查填空 1 和 2 的代码。

## 第四部分：定义分类模型

In [ ]:
class SimpleClassifier(nn.Module):
    """
    简单的二分类神经网络
    
    结构：输入(2) → 隐藏层(16) → 输出(1)
    """
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(2, 16)    # 输入2维特征，隐藏层16个神经元
        self.fc2 = nn.Linear(16, 1)    # 输出1维（二分类）
        self.sigmoid = nn.Sigmoid()     # Sigmoid 激活输出概率
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))     # ReLU 激活
        x = self.sigmoid(self.fc2(x))   # Sigmoid 输出概率
        return x


def train_model(X_train, y_train, epochs=100):
    """
    训练分类模型
    
    参数:
        X_train: 训练特征
        y_train: 训练标签
        epochs: 训练轮数
    
    返回:
        model: 训练好的模型
    """
    # 转换为 PyTorch 张量
    X = torch.FloatTensor(X_train)
    y = torch.FloatTensor(y_train).reshape(-1, 1)
    
    # 创建模型、损失函数和优化器
    model = SimpleClassifier()
    criterion = nn.BCELoss()  # 二分类交叉熵损失
    optimizer = optim.Adam(model.parameters(), lr=0.01)
    
    # 训练循环
    for epoch in range(epochs):
        optimizer.zero_grad()
        outputs = model(X)
        loss = criterion(outputs, y)
        loss.backward()
        optimizer.step()
    
    return model


def evaluate_model(model, X_test, y_test):
    """
    评估模型准确率
    
    参数:
        model: 训练好的模型
        X_test: 测试特征
        y_test: 测试标签
    
    返回:
        accuracy: 准确率
    """
    X = torch.FloatTensor(X_test)
    
    with torch.no_grad():
        predictions = model(X).numpy().flatten()
    
    # ========== 填空 3：概率转类别 ==========
    # 
    # 🎯 任务：将模型输出的概率转换为类别预测
    # 
    # 💡 提示：
    #   - 模型输出是 0-1 之间的概率值
    #   - 概率 > 0.5 预测为类别 1，否则为类别 0
    #   - 使用 (predictions > 0.5) 得到布尔数组
    #   - 使用 .astype(int) 转换为整数
    # 
    # 难度：⭐⭐☆☆☆
    # 
    # 请将 ___________ 替换为正确的代码
    
    predicted_labels = ___________  # 期望：0 或 1 的数组
    
    # ========== 填空 3 结束 ==========
    
    # 计算准确率
    accuracy = np.mean(predicted_labels == y_test)
    return accuracy

print("=" * 50)
print("🤖 模型定义完成")
print("=" * 50)
print("  模型结构: 2 → 16 → 1")
print("  激活函数: ReLU + Sigmoid")
print("  损失函数: 二分类交叉熵 (BCE)")
print("\n✓ 准备开始实验！")

## 第五部分：对比实验 - 不同投毒比例的影响

这是本实验的**核心环节**：我们将测试不同投毒比例（0% ~ 30%）对模型准确率的影响。

In [ ]:
# ========== 填空 4：设置投毒比例列表 ==========
# 
# 🎯 任务：定义要测试的投毒比例列表
# 
# 💡 提示：
#   - 包含 0.0（基准，无投毒）
#   - 逐步增加：0.05, 0.10, 0.15, 0.20, 0.30
#   - 使用 Python 列表格式
# 
# 难度：⭐☆☆☆☆
# 
# 请将 ___________ 替换为正确的代码

poison_ratios = ___________  # 期望：[0.0, 0.05, 0.10, 0.15, 0.20, 0.30]

# ========== 填空 4 结束 ==========

accuracies = []

print("=" * 60)
print("🔬 开始对比实验：不同投毒比例的影响")
print("=" * 60)
print(f"{'投毒比例':<12} | {'投毒样本数':<12} | {'测试准确率':<12}")
print("-" * 60)

for ratio in poison_ratios:
    # 对训练数据进行投毒
    y_poisoned, poison_idx = poison_labels(y_train, ratio)
    
    # 训练模型
    model = train_model(X_train, y_poisoned, epochs=100)
    
    # 在干净的测试集上评估
    acc = evaluate_model(model, X_test, y_test)
    accuracies.append(acc)
    
    # 输出结果
    status = "✓ 基准" if ratio == 0 else ("⚠ 轻微" if acc > 0.85 else "✗ 严重")
    print(f"{ratio*100:>8.1f}%    | {len(poison_idx):>8}     | {acc*100:>8.2f}%   {status}")

print("=" * 60)
print(f"\n📉 准确率下降: {(accuracies[0] - accuracies[-1])*100:.2f}% （从 0% 投毒到 30% 投毒）")
print("\n✓ 对比实验完成！")

### 🤔 思考一下

1. **观察**：投毒比例从 0% 增加到 30%，准确率发生了什么变化？
2. **分析**：为什么即使只有 5-10% 的数据被投毒，模型准确率也会明显下降？
3. **应用**：在实际场景中，攻击者只需要污染很小比例的数据就能产生影响，这说明了什么？

## 第六部分：可视化投毒效果

In [ ]:
# 绘制投毒比例与准确率的关系
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# 左图：准确率变化曲线
ax1 = axes[0]
x_percent = [r*100 for r in poison_ratios]
y_percent = [a*100 for a in accuracies]

ax1.plot(x_percent, y_percent, 'bo-', linewidth=2, markersize=10)
ax1.fill_between(x_percent, y_percent, alpha=0.3)
ax1.set_xlabel('投毒比例 (%)', fontsize=12)
ax1.set_ylabel('测试准确率 (%)', fontsize=12)
ax1.set_title('标签翻转攻击效果', fontsize=14)
ax1.grid(True, alpha=0.3)
ax1.set_ylim([40, 105])

# 添加数据标签
for i, (x, y) in enumerate(zip(x_percent, y_percent)):
    ax1.annotate(f'{y:.1f}%', (x, y), textcoords="offset points", 
                 xytext=(0, 10), ha='center', fontsize=9)

# 右图：准确率下降幅度（柱状图）
ax2 = axes[1]
baseline = accuracies[0]
drops = [(baseline - a) * 100 for a in accuracies]
colors = ['green' if d < 5 else 'orange' if d < 15 else 'red' for d in drops]

bars = ax2.bar([f"{r*100:.0f}%" for r in poison_ratios], drops, color=colors, edgecolor='black')
ax2.set_xlabel('投毒比例', fontsize=12)
ax2.set_ylabel('准确率下降 (%)', fontsize=12)
ax2.set_title('准确率下降幅度', fontsize=14)
ax2.axhline(y=5, color='orange', linestyle='--', linewidth=1.5, label='轻微影响 (5%)')
ax2.axhline(y=15, color='red', linestyle='--', linewidth=1.5, label='严重影响 (15%)')
ax2.legend(loc='upper left')
ax2.grid(True, alpha=0.3, axis='y')

# 在柱子上添加数值
for bar, drop in zip(bars, drops):
    ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5, 
             f'{drop:.1f}%', ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()

print("\n📊 图表说明：")
print("  - 左图：准确率随投毒比例增加而下降")
print("  - 右图：绿色=安全，橙色=轻微影响，红色=严重影响")

## 第七部分：可视化投毒数据分布

让我们直观地看看被投毒的样本在数据空间中的位置。

In [ ]:
# ========== 填空 5：设置可视化的投毒比例 ==========
# 
# 🎯 任务：选择一个较高的投毒比例来可视化
# 
# 💡 提示：
#   - 20% 是一个不错的选择，效果明显
#   - 使用小数表示：0.20
# 
# 难度：⭐☆☆☆☆
# 
# 请将 ___________ 替换为正确的代码

vis_ratio = ___________  # 期望：0.20

# ========== 填空 5 结束 ==========

# 对数据投毒
y_poisoned_vis, poison_idx_vis = poison_labels(y_train, vis_ratio)

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# 左图：原始干净数据
ax1 = axes[0]
ax1.scatter(X_train[y_train==0, 0], X_train[y_train==0, 1], 
            c='blue', label='类别 0', alpha=0.5, s=30)
ax1.scatter(X_train[y_train==1, 0], X_train[y_train==1, 1], 
            c='red', label='类别 1', alpha=0.5, s=30)
ax1.set_xlabel('特征 1', fontsize=12)
ax1.set_ylabel('特征 2', fontsize=12)
ax1.set_title('原始数据（干净标签）', fontsize=14)
ax1.legend()
ax1.grid(True, alpha=0.3)
ax1.axhline(y=0, color='gray', linestyle='--', alpha=0.3)
ax1.axvline(x=0, color='gray', linestyle='--', alpha=0.3)

# 右图：投毒后数据（标记被翻转的样本）
ax2 = axes[1]

# 绘制正常样本（使用投毒后的标签着色）
normal_idx = np.setdiff1d(np.arange(len(y_train)), poison_idx_vis)
ax2.scatter(X_train[normal_idx, 0], X_train[normal_idx, 1], 
            c=['blue' if y==0 else 'red' for y in y_poisoned_vis[normal_idx]], 
            alpha=0.3, s=20, label='正常样本')

# 绘制投毒样本（用醒目的标记）
ax2.scatter(X_train[poison_idx_vis, 0], X_train[poison_idx_vis, 1], 
            c='yellow', edgecolors='black', s=120, marker='X', 
            linewidths=1.5, label=f'投毒样本（标签翻转）')

ax2.set_xlabel('特征 1', fontsize=12)
ax2.set_ylabel('特征 2', fontsize=12)
ax2.set_title(f'投毒后数据（{int(vis_ratio*100)}% 标签被翻转）', fontsize=14)
ax2.legend()
ax2.grid(True, alpha=0.3)
ax2.axhline(y=0, color='gray', linestyle='--', alpha=0.3)
ax2.axvline(x=0, color='gray', linestyle='--', alpha=0.3)

plt.tight_layout()
plt.show()

print("\n📌 观察要点：")
print("  - 黄色 X 标记的点是被投毒的样本")
print("  - 这些点的标签被翻转，导致模型学习到错误的决策边界")
print(f"  - 共有 {len(poison_idx_vis)} 个样本被投毒（{vis_ratio*100:.0f}%）")

### ✅ 检查点 2

运行到这里，你应该看到：
- [ ] 左图显示干净数据，蓝色和红色清晰分离
- [ ] 右图中黄色 X 标记的投毒样本分布在两个类别中
- [ ] 投毒样本约占总数的 20%

如果看不到黄色标记，请检查填空 5 的代码。

## 📋 实验小结

### 核心收获

1. **概念**：理解了标签翻转攻击的原理——通过故意标注错误标签来污染训练数据

2. **技能**：掌握了实现标签翻转攻击的方法和评估投毒影响的实验设计

3. **思考**：认识到即使少量数据被投毒（5-10%），也能显著影响模型性能

### 关键代码回顾

```python
# 标签翻转的核心操作
n_poison = int(n_samples * poison_ratio)  # 计算投毒数量
poison_indices = np.random.choice(n_samples, n_poison, replace=False)  # 随机选择
y_poisoned[poison_indices] = 1 - y_poisoned[poison_indices]  # 翻转标签
```

### 标签翻转攻击的特点

| 优点（对攻击者） | 缺点（对攻击者） |
|-----------------|------------------|
| 实现简单，无需了解模型 | 会降低模型整体性能 |
| 成本低，只需修改标签 | 容易被检测（性能下降明显） |
| 对所有模型都有效 | 攻击不可控（随机出错） |

### 延伸思考

1. 如果你是数据收集方，如何防止众包标注中的恶意标签？
2. 标签翻转攻击 vs 后门攻击，哪个更隐蔽？（下一个实验将探讨）

## 📚 参考答案

<details>
<summary>点击展开填空参考答案</summary>

**填空 1：计算投毒样本数量**
```python
n_poison = int(n_samples * poison_ratio)
```

**填空 2：翻转标签**
```python
y_poisoned[poison_indices] = 1 - y_poisoned[poison_indices]
```

**填空 3：概率转类别**
```python
predicted_labels = (predictions > 0.5).astype(int)
```

**填空 4：设置投毒比例列表**
```python
poison_ratios = [0.0, 0.05, 0.10, 0.15, 0.20, 0.30]
```

**填空 5：设置可视化的投毒比例**
```python
vis_ratio = 0.20
```

</details>

In [ ]:
# 实验结束，清理显存（可选）
# 如果要继续进行下一个实验，可以运行这段代码释放显存

import gc
import torch

# 删除大对象（如果存在）
for var_name in ['model', 'tokenizer']:
    if var_name in globals():
        del globals()[var_name]

# 垃圾回收
gc.collect()

# 清理 CUDA 显存
if torch.cuda.is_available():
    torch.cuda.empty_cache()

print("✓ 资源已清理")

⚠️ 重要提醒

实验完成后，请记得：

1. **释放资源**：如果你在 Cloud Studio 使用了 GPU，建议点击右上角的“停止”按钮，停止运行，避免持续占用免费额度。
2. **保存结果**：如需提交或留存，请下载并保存本 notebook 文件。